In [1]:
# This code shows how to verify our results using PEPit 
# To run this code, you need to run the code in 2. PEPit_verification_R.ipynb first to get the input that will be fed into PEPit
import PEPit

In [2]:
from PEPit import *
from PEPit.functions import SmoothStronglyConvexFunction

In [3]:
def NCG_PEP(L, mu, gamma, beta, c0, eta, n, rho_LB, rho_UB,  verbose=1, dim_red = "none"):
    
    # dim_red = "none", means we will call default PEPit
    # dim_red = "trace", means we will call PEPit with trace heuristics that tries to find low dimension solutions to the SDP. 
    # dim_red = "logdet10", means we will call PEPit with logdet heuristics that tries to find low dimension solutions to the SDP with 10 iterations.
    # Here, you can change the number to anything you like, such as "logdet100" etc 
    
    # Instantiate PEP
    problem = PEP()
    
    # Declare a smooth strongly convex function
    f = problem.declare_function(SmoothStronglyConvexFunction, mu=mu, L=L)
    
    # Start by defining its unique optimal point xs = x_* and 
    # corresponding function value fs = f_*
    xs = f.stationary_point()
    fs = f.value(xs)
    
    # Then define the starting point x0 of the algorithm as well as corresponding gradient and function value g0 and f0
    x0 = problem.set_initial_point()
    g0, f0 = f.oracle(x0)
    
    d0 = Point()
    d_new = d0
    x_new = x0
    g_new = g0
    
    problem.add_constraint((d0) ** 2 <= c0*(g0) ** 2)

    problem.add_constraint((d0) ** 2 >= (g0) ** 2)

    problem.add_constraint(g0 * d0 == (g0) ** 2)
    
    # Set the initial constraint that is the difference between f0 and f_*
    problem.set_initial_condition(f0 - fs == 1)
    
    for i in range(n):
        print(i)
        d_old = d_new
        x_old = x_new
        g_old = g_new
        x_new = x_old - gamma[i] * d_old
        g_new = f.gradient(x_new)
        d_new = g_new + beta[i] * d_old
        problem.add_constraint(g_new * d_old == 0)
        problem.add_constraint(g_new * (x_old-x_new) == 0)
        problem.add_constraint(g_old * d_old == g_old ** 2)
        if i < n-1:
            print('Adding beta[i] constraint')
            problem.add_constraint(beta[i] * g_old ** 2 == g_new **2 - eta * (g_new * g_old))
        
      
    # Set the performance metric to the function value accuracy
    problem.set_performance_metric(f.value(x_new) - fs)   
    
    # Solve the PEP
    verbose = 1
    pepit_verbose = max(verbose, 0)
    
    if dim_red == "none":
        pepit_rho = problem.solve(verbose=pepit_verbose)
    elif dim_red == "trace":
        pepit_rho = problem.solve(verbose=pepit_verbose, dimension_reduction_heuristic=dim_red)
    else:
        pepit_rho = problem.solve(verbose=pepit_verbose, dimension_reduction_heuristic=dim_red)

        
    # Print conclusion if required
    if verbose != -1:
        print('\t*** Output  ***')
        print('\t*** NOTE: PEPIt can verify the lower bound only  ***')
        print('\t*** LOWER BOUND ***')
        print('\t####################################')
        print("\tPEPit guarantee: (Lower bound)\t\t\t\t f(x_",n,")-f_* >= {:.6} (f(x_0)-f_*)".format(pepit_rho))
        print("\tNCG-PEP guarantee: (Lower bound)\t\t\t f(x_",n,")-f_* >= {:.6} (f(x_0)-f_*)".format(rho_LB))
        print('\t####################################')
        print('\t*** UPPER BOUND ***')
        print('\t####################################')
        print("\tNCG-PEP guarantee: (Upper bound)\t\t\t f(x_",n,")-f_* <= {:.6} (f(x_0)-f_*)".format(rho_UB))
        print('\t####################################')
        
    return pepit_rho   

In [4]:
# PRP example
# ===========

## Parameter values
L = 1;

mu = 0.5
n = 1
beta = [0.15999320011971394]
gamma = [1.0666876040969064]
rho_LB = 0.35999999196299937
rho_UB = 0.3600000412215619
########################################################################################################################
# PLEASE NOTE THAT: eta = 1 means PRP, eta = 0 means FR, please change it depending on which data set you are loading
########################################################################################################################
eta = 1

# factor for PRP
c0 = (L+mu)**2/(4*mu*L) 

In [5]:
rho_PEP = NCG_PEP(L, mu, gamma, beta, c0, eta, n, rho_LB, rho_UB,  verbose=1, dim_red = "none")
# options for dim_red are "none", "trace" or "logdetX", where X is some integer such as 10, 11 etc

0
(PEPit) Setting up the problem: size of the main PSD matrix: 5x5
(PEPit) Setting up the problem: performance measure is minimum of 1 element(s)
(PEPit) Setting up the problem: Adding initial conditions and general constraints ...
(PEPit) Setting up the problem: initial conditions and general constraints (7 constraint(s) added)
(PEPit) Setting up the problem: interpolation conditions for 1 function(s)
		 function 1 : Adding 6 scalar constraint(s) ...
		 function 1 : 6 scalar constraint(s) added
(PEPit) Setting up the problem: constraints for 0 function(s)
(PEPit) Compiling SDP
(PEPit) Calling SDP solver
(PEPit) Solver status: optimal (solver: SCS); optimal value: 0.3599800559467299
	*** Output  ***
	*** NOTE: PEPIt can verify the lower bound only  ***
	*** LOWER BOUND ***
	####################################
	PEPit guarantee: (Lower bound)				 f(x_ 1 )-f_* >= 0.35998 (f(x_0)-f_*)
	NCG-PEP guarantee: (Lower bound)			 f(x_ 1 )-f_* >= 0.36 (f(x_0)-f_*)
	#################################